In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim

ModuleNotFoundError: No module named 'torch'

In [96]:
class classifier_LSTM(nn.Module):

    def __init__(self,
                 relup,
                 input_size,
                 lstm_layers,
                 lstm_size,
                 output1_size,
                 output2_size,
                 GPUindex):
        super(classifier_LSTM, self).__init__()
        self.relup = relup
        self.lstm_layers = lstm_layers
        self.lstm_size = lstm_size
        self.GPUindex = GPUindex
        self.lstm = nn.LSTM(
            input_size, lstm_size, num_layers = 1, batch_first = True)
        self.output1 = nn.Linear(lstm_size, output1_size)
        self.relu = nn.ReLU()
        if output2_size is None:
            self.output2 = None
        else:
            self.output2 = nn.Linear(lstm_size, output2_size)

    def forward(self, x):
        x = torch.tensor(x)
        batch_size = x.size(0)
        lstm_init = (torch.zeros(self.lstm_layers, batch_size, self.lstm_size),
                     torch.zeros(self.lstm_layers, batch_size, self.lstm_size))
        if x.is_cuda:
            lstm_init = (lstm_init[0].cuda(self.GPUindex),
                         lstm_init[0].cuda(self.GPUindex))
        lstm_init = (Variable(lstm_init[0]), Variable(lstm_init[1]))
        x = self.lstm(x, lstm_init)[0][:, -1, :]
        x = self.output1(x)
        if self.relup:
             x = self.relu(x)
        if self.output2 is not None:
            x = self.output2(x)
        return x

In [4]:
'''if classifier=="LSTM":
                if kind=="from-scratch":
                    output_size = 128
                if kind=="incremental":
                    output_size = 128
                if kind=="no-model-file":
                    output_size = 128
                net = classifier_LSTM(
                    True,
                    input_size = channel,
                    lstm_layers = 1,
                    lstm_size = 128,
                    output1_size = 128,
                    output2_size = None,
                    GPUindex = GPUindex)
            elif classifier=="LSTM1":
                if kind=="from-scratch":
                    output_size = 128
                if kind=="incremental":
                    output_size = 128
                if kind=="no-model-file":
                    output_size = 128
                net = classifier_LSTM(
                    False,
                    input_size = channel,
                    lstm_layers = 1,
                    lstm_size = 128,
                    output1_size = 128,
                    output2_size = None,
                    GPUindex = GPUindex)
            elif classifier=="LSTM2":
                if kind=="from-scratch":
                    output_size = len(classes)
                if kind=="incremental":
                    output_size = total
                if kind=="no-model-file":
                    output_size = len(classes)
                net = classifier_LSTM(
                    False,
                    input_size = channel,
                    lstm_layers = 1,
                    lstm_size = 128,
                    output1_size = output_size,
                    output2_size = None,
                    GPUindex = GPUindex)
            elif classifier=="LSTM3":
                if kind=="from-scratch":
                    output_size = len(classes)
                if kind=="incremental":
                    output_size = total
                if kind=="no-model-file":
                    output_size = len(classes)
                net = classifier_LSTM(
                    True,
                    input_size = channel,
                    lstm_layers = 1,
                    lstm_size = 128,
                    output1_size = output_size,
                    output2_size = None,
                    GPUindex = GPUindex)
            elif classifier=="LSTM4":
                if kind=="from-scratch":
                    output_size = len(classes)
                if kind=="incremental":
                    output_size = total
                if kind=="no-model-file":
                    output_size = len(classes)
                net = classifier_LSTM(
                    True,
                    input_size = channel,
                    lstm_layers = 1,
                    lstm_size = 128,
                    output1_size = 128,
                    output2_size = output_size,
                    GPUindex = GPUindex)
'''

In [97]:
model=classifier_LSTM(relup=True,input_size = 96,
                    lstm_layers = 1,
                    lstm_size = 128,
                    output1_size = 128,
                    output2_size = None,
                    GPUindex = 1)


In [98]:
from pytorch_model_summary import summary

In [99]:
summary(model,(98304,96))

RuntimeError: For unbatched 2-D input, hx and cx should also be 2-D but got (3-D, 3-D) tensors